In [ ]:
import tensorflow_data_validation as tfdv
import pandas as pd
import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from pandas.api.types import is_numeric_dtype
from collections import Counter,defaultdict

from tensorflow_data_validation.utils.schema_util import get_feature,get_categorical_features,get_categorical_numeric_features,is_categorical_feature,get_multivalent_features
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
testdf = pd.read_csv('../data_test.csv')
test_metadata = pd.read_csv('../RawCSV/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

print(test_merged)
y_true = test_merged.y_act.values.tolist()
print(y_true)
print(len(y_true))

In [ ]:
# test_merged.head(400)

In [ ]:
cntExceptions = 0
y_tfdv = [0]*1985
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in test_merged.iterrows():
#     print(index)        
    if index%100==0: 
#         print(y_tfdv)
        print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../RawCSV/' + row['name']
    
    print(csv_name)
    print(col)
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')

    try:
        df_col = df[[col]]
    except KeyError:
        print('Exception Index:')
        print(index)
        y_tfdv[index]=1
        exception_indices.append(row)
        cntExceptions += 1
        continue
        
    st_option = tfdv.StatsOptions(enable_semantic_domain_stats=True)
    
    try:
        stats = tfdv.generate_statistics_from_dataframe(df_col,stats_options=st_option)
    except:
        print('Exception Index:')        
        print(index)
        y_tfdv[index]=1
        exception_indices.append(index)        
        continue
        
    schema = tfdv.infer_schema(statistics=stats)
    
    categ_lst = get_categorical_features(schema)
    for x in categ_lst: 
        print('Categorical')
        y_tfdv[index]=1
        break
    
    xc = schema.feature
    for x in xc:
        cnt_NLD = str(x).count('natural_language_domain')
        cnt_TD = str(x).count('time_domain')

        if cnt_NLD:
            print('Sentence')
            y_tfdv[index] = 3
        if cnt_TD:
            print('Time')
            y_tfdv[index] = 2   
        
    print()

In [ ]:
print(accuracy_score(y_true, y_tfdv))
print(confusion_matrix(y_true, y_tfdv))